In [2]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [13]:
STORM_NUMBER = 10
WINTER = "2024-2025"
TITLE = "7-8 February 2025"
SUBTITLE = "8 AM 9 February 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 2, 8, 0)
ets = datetime(2025, 2, 9, 18)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [4]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [5]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [6]:
df

,geo,lon,lat,val,nwsli,used_for_analysis,plotme,state,xcell,ycell
0,POINT (260227.691 -542858.648),-96.943658,40.073237,0,Z1,True,False,Z,0,0
1,POINT (350227.691 -542858.648),-95.887852,40.038178,0,Z2,True,False,Z,3,0
2,POINT (440227.691 -542858.648),-94.833132,39.992809,0,Z3,True,False,Z,6,0
3,POINT (530227.691 -542858.648),-93.779772,39.937150,0,Z4,True,False,Z,9,0
4,POINT (620227.691 -542858.648),-92.728043,39.871226,0,Z5,True,False,Z,12,0
5,POINT (710227.691 -542858.648),-91.678210,39.795067,0,Z6,True,False,Z,15,0
6,POINT (800227.691 -542858.648),-90.630535,39.708707,0,Z7,True,False,Z,18,0
7,POINT (260227.691 -452858.648),-96.905698,40.882564,0,Z8,True,False,Z,0,3
8,POINT (350227.691 -452858.648),-95.836836,40.846952,0,Z9,True,False,Z,3,3
9,POINT (440227.691 -452858.648),-94.769122,40.800868,0,Z10,True,False,Z,6,3


In [15]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'DVNI4',
        'IA-CY-6',
        'IA-WB-3',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=2)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250209.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
1342    IA  ARX  3.2 -92.920000  43.380000  POINT (571353.861 -155313.144)   
1978    IA  FSD  1.0 -96.480000  43.000000   POINT (286189.57 -216214.523)   
2049    IA  ARX  0.8 -92.320000  43.060000  POINT (622893.443 -186394.507)   
2118    IA  FSD  0.5 -96.270000  42.810000  POINT (304194.589 -236564.897)   
2138    IA  FSD  0.4 -96.050000  43.000000  POINT (321118.158 -214616.722)   
...    ...  ...  ...        ...        ...                             ...   
4312    IA  DVN  0.0 -91.331382  41.510436  POINT (720480.747 -349937.184)   
4313    IA  DVN  0.0 -90.587065  41.644656  POINT (780366.795 -328280.485)   
4314    IA  DVN  0.0 -90.495000  41.728000  POINT (786918.595 -318180.756)   
4315    IA  DVN  0.0 -90.632791  41.538730  POINT (777895.555 -340434.051)   
4316    IA  DVN  0.0 -90.432900  41.576900   POINT (793925.198 -334298.19)   

      used_for_analysis     nwsli  plotme    source  xcell  yce

## 